# 여행가이드 챗봇

## RAG 실습 준비

In [9]:
!pip install langchain_community langchain_text_splitters langchain_openai langchain_chroma

In [10]:
import os

from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_chroma import Chroma
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

In [11]:
# 0. 초기설정
import os
from getpass import getpass

# OpenAI API 키 입력
openai_api_key = getpass('Enter your OpenAI API key: ')

# OpenAI API 키 저장
os.environ['OPENAI_API_KEY'] = openai_api_key

In [12]:
# 가상 데이터 생성(여행가이드 문서)
jeju_content = """
[제주도 맛집 & 여행 가이드]
1. 흑돼지 거리: 제주시 건입동에 위치. 저녁 7시 이후에는 웨이팅이 1시간 이상 발생할 수 있음. 
   추천 메뉴는 흑돼지 오겹살(200g 2만원). 멜젓에 찍어 먹는 것이 특징.
   
2. 우도 땅콩 아이스크림: 우도 검멀레 해변 앞이 원조. 가격은 5,000원. 
   고소한 땅콩 가루가 듬뿍 뿌려져 있음. 배 시간 때문에 오후 4시 이전에 가는 것을 추천.
   
3. 성산일출봉: 입장료 5,000원. 왕복 소요 시간 50분. 
   매월 첫째 주 월요일은 휴관. 새벽 일출을 보려면 전날 근처 숙소 예약 필수.
"""

# 파일 저장
with open("jeju_guide.txt", "w", encoding="utf-8") as f:
    f.write(jeju_content)

print("[준비완료] 여행 가이드 문서가 생성되었습니다.")

[준비완료] 여행 가이드 문서가 생성되었습니다.


## RAG 실습 시작 - LangChain 사용

### 문서 로딩

In [14]:
# 1. 문서 로딩
print("\n 문서를 불러옵니다...")
loader = TextLoader("jeju_guide.txt", encoding="utf-8")
docs = loader.load()
print(f"   -> {len(docs[0].page_content)}자")


 문서를 불러옵니다...
   -> 315자


In [15]:
# 2. 문서 분할
print("\n 문서를 작은 조각(Chunk)으로 나눕니다...")
text_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=20)
splits = text_splitter.split_documents(docs)
print(f"\n 총 {len(splits)}개의 조각(Chunk)로 분할되었습니다.")


 문서를 작은 조각(Chunk)으로 나눕니다...

 총 4개의 조각(Chunk)로 분할되었습니다.


In [16]:
# 3. 임베딩 & 벡터 저장소 생성
print("\n 문서를 벡터로 변환하여 저장합니다...")
vectorstore = Chroma.from_documents(
    documents=splits,
    embedding=OpenAIEmbeddings(model="text-embedding-3-small")
)

print("   -> Chroma DB에 저장완료!")


 문서를 벡터로 변환하여 저장합니다...
   -> Chroma DB에 저장완료!


In [17]:
# 4. Retriever 설정 & Generation
print("\n RAG 체인을 생성합니다...")

retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)


 RAG 체인을 생성합니다...


In [19]:
# 프롬프트 템플릿 작성
template = """
당신은 제주도 여행 가이드 담당자압니다.
아래 [가이드]를 참고하여 여행자의 질문에 친절하게 답해주세요.
가이드에 없는 내용은 별로의 검색엔진을 이용하도록 안내해 주세요.

[가이드]:
{context}

[질문]:
{question}
"""
prompt = PromptTemplate.from_template(template)

In [20]:
# 체인 연결
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [21]:
# 5. 실행
question = "흑돼지 먹으려면 어디로 가야 해? 그리고 성산일출봉 입장료는 얼마야?"

print(f"\n 질문: {question}")
print("AI 답변 생성중...\n")

response = rag_chain.invoke(question)

print(response)



 질문: 흑돼지 먹으려면 어디로 가야 해? 그리고 성산일출봉 입장료는 얼마야?
AI 답변 생성중...

흑돼지를 드시려면 제주시 건입동에 위치한 흑돼지 거리를 추천합니다. 저녁 7시 이후에는 웨이팅이 1시간 이상 발생할 수 있으니, 미리 가시는 것이 좋습니다.

성산일출봉의 입장료는 5,000원입니다. 왕복 소요 시간은 약 50분 정도 걸립니다. 즐거운 여행 되세요!


In [25]:
# 5. 실행
question = "성산일출봉을 편도로 가면 시간이 얼마나 걸릴까?"

print(f"\n 질문: {question}")
print("AI 답변 생성중...\n")

response = rag_chain.invoke(question)

print(response)


 질문: 성산일출봉을 편도로 가면 시간이 얼마나 걸릴까?
AI 답변 생성중...

성산일출봉은 왕복 소요 시간이 약 50분입니다. 편도로 가는 경우에는 대략 25분 정도 걸릴 것으로 예상됩니다. 즐거운 여행 되세요!


In [23]:
# 5. 실행
question = "흑돼지 2인분과 우동 땅콩아이스트림 2개를 먹었는데 합계 금액은 얼마야?"

print(f"\n 질문: {question}")
print("AI 답변 생성중...\n")

response = rag_chain.invoke(question)

print(response)


 질문: 흑돼지 2인분과 우동 땅콩아이스트림 2개를 먹었는데 합계 금액은 얼마야?
AI 답변 생성중...

흑돼지 오겹살 2인분(200g 기준, 1인분 2만원)과 우도 땅콩 아이스크림 2개(가격 5,000원)로 계산해보면:

- 흑돼지 오겹살 2인분: 2만원 x 2 = 4만원
- 우도 땅콩 아이스크림 2개: 5,000원 x 2 = 1만원

따라서, 총 합계 금액은 4만원 + 1만원 = 5만원입니다. 맛있게 드세요!
